# Week 3 Exercise 1: Create a patient dataset

### Future To Do:
- Float columns should be converted to ints where needed

In [1]:
import sqlite3 as sql
import pandas as pd
import numpy as np
from datetime import date

In [2]:
# DATE to use for diffs
enddate = date(2022, 10, 1) # Date chosen for when data created

In [3]:
# Define helpful functions

# Calcuate age
def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [4]:
# Connect to database (need the current working directory for systems that treat the local path as virtual)
import os
con = sql.connect(os.path.join(os.getcwd(), "synthea_and_county_ga.db"))

In [5]:
# Start the dataframe with basic patient information
# (include fips to prepare for joining with county data)
df = pd.read_sql_query("""\
    select id, 
        first, 
        last, 
        city, 
        state, 
        county, 
        ahrf.fips, 
        lat, 
        lon, 
        birthdate, 
        marital, 
        race, 
        ethnicity, 
        gender, 
        healthcare_expenses as cost_lifetime, 
        income
    from patients
    left join us_counties_ahrf_2021 ahrf on (patients.county = (ahrf.county_name || ' County')) and (patients.state = ahrf.state_name)
    order by first, last, birthdate
""", con)

In [6]:
# Quick settings and clean-up

# Display all columns instead of only a selected set
pd.options.display.max_columns = None

# convert column names to lowercase
df = df.rename(columns=str.lower)

# Set the index to the id, rather than having pandas create a autocount index
df = df.set_index('id')

In [7]:
df.shape

(22632, 15)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22632 entries, 53e40e98-c764-53a4-aaf6-6318a3c3c95d to 9fdfb702-0f46-8899-fe8c-363733532bb6
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   first          22632 non-null  object 
 1   last           22632 non-null  object 
 2   city           22632 non-null  object 
 3   state          22632 non-null  object 
 4   county         22632 non-null  object 
 5   fips           22632 non-null  int64  
 6   lat            22632 non-null  float64
 7   lon            22632 non-null  float64
 8   birthdate      22632 non-null  object 
 9   marital        14505 non-null  object 
 10  race           22632 non-null  object 
 11  ethnicity      22632 non-null  object 
 12  gender         22632 non-null  object 
 13  cost_lifetime  22632 non-null  float64
 14  income         22632 non-null  int64  
dtypes: float64(3), int64(2), object(10)
memory usage: 2.8+ MB


## Calcuated / Summarized Features (e.g., age, etc.)

In [9]:
# Calc age per row
df['birthdate'] = pd.to_datetime(df['birthdate'])
df['age'] = df['birthdate'].apply(calculate_age)

In [10]:
# Marital: Replace missing values with 'unknown'
df['marital'] = df['marital'].fillna('unknown').astype('str')

In [11]:
# Type of insurance (from most recent encounter... did not consider payer transitions)
df_temp = pd.read_sql_query("""\
    select distinct p.Id, pay.Name as InsuranceType
    from patients p
    left outer join encounters e on p.Id = e.PATIENT
    left outer join payers pay on e.payer = pay.Id
    group by p.Id
    having max(e.start)
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')

# Categorize insurance type by replacing names with with types
df_temp['insurancetype'].replace('Blue Cross Blue Shield', 'commercial', inplace=True)
df_temp['insurancetype'].replace('Aetna', 'commercial', inplace=True)
df_temp['insurancetype'].replace('Anthem', 'commercial', inplace=True)
df_temp['insurancetype'].replace('Cigna Health', 'commercial', inplace=True)
df_temp['insurancetype'].replace('Humana', 'commercial', inplace=True)
df_temp['insurancetype'].replace('UnitedHealthcare', 'commercial', inplace=True)

df_temp['insurancetype'].replace('Medicare', 'medicare', inplace=True)
df_temp['insurancetype'].replace('Medicaid', 'medicaid', inplace=True)
df_temp['insurancetype'].replace('Dual Eligible', 'dualelig', inplace=True)

df_temp['insurancetype'].replace('NO_INSURANCE', 'noins', inplace=True)

df = pd.merge(df, df_temp, on='id', how='outer')

df['insurancetype'] = df['insurancetype'].fillna('unknown').astype('str')

In [12]:
# Education
# From findings

df['education'] = 'unknown' # init an education feature, filled with unknown

dict_edu_codes = {'5251000175109' : 'high_sch_equiv', 
                  '224299000' : 'post_sec',
                  '224295006' : 'pri_only'}

for key in dict_edu_codes:
    sql = """
        select p.id
        from patients p
        left outer join conditions c on p.id = c.PATIENT
        where c.code in ('{0}')
    """.format(key)
        
    df_temp = pd.read_sql_query(sql, con)
    
    df_temp = df_temp.rename(columns=str.lower)
    df_temp = df_temp.set_index('id')
    
    df_temp = df_temp.loc[df_temp.index]
    df_temp['education'] = dict_edu_codes[key]

    df.update(df_temp)

## "Number of" Features (e.g., number of allergies, etc.)

In [13]:
# Add numof_ columns for all relevant, clinical tables
list_tables = ['allergies', 'careplans', 'devices', 'medications']

for tbl in list_tables:
    sql = """
        select p.Id, count(a.PATIENT) as numof_{0}
        from patients p
        left outer join {0} a on p.Id = a.PATIENT
        where stop is null --active
        group by p.Id
    """.format(tbl)
    
    df_temp = pd.read_sql_query(sql, con)
    
    df_temp = df_temp.rename(columns=str.lower)
    df_temp = df_temp.set_index('id')

    df = pd.merge(df, df_temp, on='id', how='outer')
    
    df['numof_' + tbl] = df['numof_' + tbl].fillna(0).astype(int)
    

In [14]:
# Number of Procedures (open or within last two years)
sql = f"""
    select p.Id, count(pr.PATIENT) as numof_procedures_2yr
    from patients p
    left outer join procedures pr on p.Id = pr.PATIENT
    where (julianday('{enddate}') - julianday(start) <= 730) OR stop is null -- active or 2 years or less
    group by p.Id
"""

df_temp = pd.read_sql_query(sql, con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')

df = pd.merge(df, df_temp, on='id', how='outer')
df['numof_procedures_2yr'] = df['numof_procedures_2yr'].fillna(0).astype(int)

In [15]:
# Number of active medical conditions
# (Note: Drop anything that is a non-medical diagnoses for this count... for some reason this data has mixed diagnoses with social determinants)
df_temp = pd.read_sql_query("""\
    select p.Id, count(p.Id) as numof_med_conds
    from patients p
    left outer join conditions c on p.Id = c.PATIENT
    where c.patient is null 
        or (
            c.stop is null and (
                c.code is null or c.code not in (
                    '160903007', --Full-time employment (finding)
                    '266948004', --Has a criminal record (finding)
                    '32911000',  --Homeless (finding)
                    '105531004', --Housing unsatisfactory (finding)
                    '713458007', --Lack of access to transportation (finding)
                    '423315002', --Limited social contact (finding)
                    '741062008', --Not in labor force (finding)
                    '224295006', --Only received primary school education (finding)
                    '160904001', --Part-time employment (finding)
                    '224299000', --Received higher education (finding)
                    '424393004', --Reports of violence in the environment (finding)
                    '160968000', --Risk activity involvement (finding)
                    '224355006', --Served in armed forces (finding)
                    '422650009', --Social isolation (finding)
                    '160701002', --Social migrant (finding)
                    '266934004', --Transport problems (finding)
                    '73438004',  --Unemployed (finding)
                    '706893006'  --Victim of intimate partner abuse (finding)
                )
            )
        )
    group by p.Id
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')

df = pd.merge(df, df_temp, on='id', how='outer')
df['numof_med_conds'] = df['numof_med_conds'].fillna(0).astype(int)

In [16]:
# Number of active social determinants (associated with challenges, not benefits)
df_temp = pd.read_sql_query("""\
    select p.Id, count(p.Id) as numof_soc_challs
    from patients p
    left outer join conditions c on p.Id = c.PATIENT
    where c.patient is null 
        or (
            c.stop is null and (
                c.code is null or c.code in (
                    '266948004', --Has a criminal record (finding)
                    '32911000',  --Homeless (finding)
                    '105531004', --Housing unsatisfactory (finding)
                    '713458007', --Lack of access to transportation (finding)
                    '423315002', --Limited social contact (finding)
                    '741062008', --Not in labor force (finding)
                    '224295006', --Only received primary school education (finding)
                    '424393004', --Reports of violence in the environment (finding)
                    '160968000', --Risk activity involvement (finding)
                    '224355006', --Served in armed forces (finding)
                    '422650009', --Social isolation (finding)
                    '160701002', --Social migrant (finding)
                    '266934004', --Transport problems (finding)
                    '73438004',  --Unemployed (finding)
                    '706893006'  --Victim of intimate partner abuse (finding)
                )
            )
        )
    group by p.Id
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')

df = pd.merge(df, df_temp, on='id', how='outer')
df['numof_soc_challs'] = df['numof_soc_challs'].fillna(0).astype(int)

In [17]:
# Number of Encounters: Step 1 of 2

# The types of encounters change sometimes, so...
# ...first get a list of all the encounter types, create them as columns, 0 filled
df_temp = pd.read_sql_query("""\
    select distinct encounterclass
    from encounters
    order by encounterclass
""", con)

df_temp = df_temp.rename(columns=str.lower)
list_encounter_classes = df_temp['encounterclass']

df = df.assign(**dict.fromkeys('numof_enc_' + list_encounter_classes + '_2yr', 0))

In [18]:
# Number of Encounters: Step 2 of 2

# ...then update the 'numof_enc_' per patient per type of encounter
df_temp = pd.read_sql_query(f"""\
    select p.Id, e.EncounterClass, count(e.Id) as numof_enc
    from patients p
    left outer join encounters e on p.Id = e.PATIENT
    where ((julianday('{enddate}') - julianday(start) <= 730) or stop is null) -- 2 years or less or active
    group by p.Id, e.EncounterClass
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')

df_temp = pd.pivot_table(df_temp, values='numof_enc', index=['id'], columns=['encounterclass'], aggfunc=np.sum, fill_value=0)

df_temp = df_temp.add_prefix('numof_enc_')
df_temp = df_temp.add_suffix('_2yr')

df.update(df_temp)

## Clean-up before continuing

In [19]:
# Zero fill and convert floats to ints
df_temp = df.select_dtypes(include=['float64'])

# Exceptions... want to keep these as a decimal
df_temp = df_temp.drop(columns=['lat', 'lon'], axis=1) 

# Convert to ints
df[df_temp.columns] = df[df_temp.columns].fillna(0).astype(int)

# Quality Meaures

## Visit information for Wellness, Emergency, and Inpatient

In [20]:
# Wellness visit: Most recent
df_temp = pd.read_sql_query("""\
    select distinct p.id, max(e.start) as enc_wellness_visit_most_recent
    from patients p
    left outer join encounters e on p.id = e.patient
    where e.encounterclass = 'wellness'
    group by p.id
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')
df_temp['enc_wellness_visit_most_recent'] = df_temp['enc_wellness_visit_most_recent'].astype('datetime64[ns]') # convert from object to date

df_temp['enc_wellness_visit_most_recent_dayssince'] = (pd.to_datetime(enddate) - pd.to_datetime(df_temp['enc_wellness_visit_most_recent'])).dt.days

df = pd.merge(df, df_temp, on='id', how='outer')

In [21]:
# Well visit within past 18 months (usually 1 year, but account for delays)
df['enc_wellness_visit_within18months'] = False
df.loc[df['enc_wellness_visit_most_recent_dayssince'] < 548, 'enc_wellness_visit_within18months'] = True

In [22]:
# ED Visit: Most recent
df_temp = pd.read_sql_query("""\
    select distinct p.id, max(e.start) as enc_ed_visit_most_recent
    from patients p
    left outer join encounters e on p.id = e.patient
    where e.encounterclass = 'emergency'
    group by p.id
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')
df_temp['enc_ed_visit_most_recent'] = df_temp['enc_ed_visit_most_recent'].astype('datetime64[ns]') # convert from object to date

df_temp['enc_ed_visit_most_recent_dayssince'] = (pd.to_datetime(enddate) - pd.to_datetime(df_temp['enc_ed_visit_most_recent'])).dt.days

df = pd.merge(df, df_temp, on='id', how='outer')

In [23]:
# Inpatient Visit: Most recent
df_temp = pd.read_sql_query("""\
    select distinct p.id, max(e.start) as enc_inpatient_visit_most_recent
    from patients p
    left outer join encounters e on p.id = e.patient
    where e.encounterclass = 'inpatient'
    group by p.id
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')
df_temp['enc_inpatient_visit_most_recent'] = df_temp['enc_inpatient_visit_most_recent'].astype('datetime64[ns]') # convert from object to date

df_temp['enc_inpatient_visit_most_recent_dayssince'] = (pd.to_datetime(enddate) - pd.to_datetime(df_temp['enc_inpatient_visit_most_recent'])).dt.days

df = pd.merge(df, df_temp, on='id', how='outer')

## Quality of Health: Observations and Conditions

In [24]:
# Obs BMI: Step 1: Most recent BMI value (most recent with date)
# NOTE: Rather than do the entire CQM measure (e.g., within range, follow-up, etc.) this code just looks for a BMI value
# Source: https://qpp.cms.gov/docs/QPP_quality_measure_specifications/CQM-Measures/2019_Measure_128_MIPSCQM.pdf
df_temp = pd.read_sql_query("""\
    select distinct p.Id, value as obs_bmi_most_recent, date as obs_bmi_most_recent_date
    from patients p
    left outer join observations o ON p.Id = o.patient
    where o.code in (
        '39156-5' -- BMI
    )
    group by p.Id
    having max(date)
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')
df_temp['obs_bmi_most_recent_date'] = df_temp['obs_bmi_most_recent_date'].astype('datetime64[ns]') # convert from object to date

df = pd.merge(df, df_temp, on='id', how='outer')

In [25]:
# Obs BMI: Step 2: See if "condition" is marked as 30+ (different table)
df_temp = pd.read_sql_query("""\
    select distinct p.Id, 1 as cond_bmi_30plus, start as cond_bmi_30plus_start
    from patients p
    inner join conditions c on p.Id = c.PATIENT
    where c.stop is null -- Active condition
        and c.code IN (
            '162864005', -- Body mass index 30+ - obesity (finding)
            '408512008'  -- Body mass index 40+ - severely obese (finding)
    )
    group by p.Id
    having max(start)
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')
df_temp['cond_bmi_30plus_start'] = df_temp['cond_bmi_30plus_start'].astype('datetime64[ns]') # convert from object to date

df_temp['cond_bmi_30plus_start_dayssince'] = (pd.to_datetime(enddate) - pd.to_datetime(df_temp['cond_bmi_30plus_start'])).dt.days
df_temp['cond_bmi_30plus_start_dayssince'] = df_temp['cond_bmi_30plus_start_dayssince'].fillna(0).astype(int)


df = pd.merge(df, df_temp, on='id', how='outer')
df['cond_bmi_30plus'] = df['cond_bmi_30plus'].fillna(0).astype(int)
df['cond_bmi_30plus'] = df['cond_bmi_30plus'].astype(bool) # Easier to deal with T/F in Tableau

In [26]:
# Cond: Diabetes or prediabetes or metabolic (active only)
# NOTE: "prediabetes" is not part of CQM, but included here
df_temp = pd.read_sql_query("""\
    select distinct p.Id, 1 as cond_diab, start as cond_diab_start
    from patients p
    inner join conditions c on p.Id = c.PATIENT
    where c.stop is null -- Active condition
        and c.code in (
            '44054006', -- Diabetes
            '127013003', -- Diabetic renal disease (disorder)
            '422034002', -- Diabetic retinopathy associated with type II diabetes mellitus (disorder)
            '161622006', -- History of lower limb amputation (situation)
            '80394007', -- Hyperglycemia (disorder)
            '302870006', -- Hypertriglyceridemia (disorder)
            '97331000119101', -- Macular edema and retinopathy due to type 2 diabetes mellitus (disorder)
            '237602007', -- Metabolic syndrome X (disorder)
            '90781000119102', -- Microalbuminuria due to type 2 diabetes mellitus (disorder)
            '368581000119106', -- Neuropathy due to type 2 diabetes mellitus (disorder)
            '1551000119108', -- Nonproliferative diabetic retinopathy due to type 2 diabetes mellitus (disorder)
            '15777000', -- Prediabetes
            '1501000119109', -- Proliferative diabetic retinopathy due to type II diabetes mellitus (disorder)
            '157141000119108' -- Proteinuria due to type 2 diabetes mellitus (disorder)
        )
    group by p.Id
    having min(start)
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')
df_temp['cond_diab_start'] = df_temp['cond_diab_start'].astype('datetime64[ns]') # convert from object to date

df_temp['cond_diab_start_dayssince'] = (pd.to_datetime(enddate) - pd.to_datetime(df_temp['cond_diab_start'])).dt.days
df_temp['cond_diab_start_dayssince'] = df_temp['cond_diab_start_dayssince'].fillna(0).astype(int)

df = pd.merge(df, df_temp, on='id', how='outer')
df['cond_diab'] = df['cond_diab'].fillna(0).astype(int)
df['cond_diab'] = df['cond_diab'].astype(bool)

In [27]:
# Obs Hba1c: Step 2: hba1c measurement (most recent)
# Source: https://qpp.cms.gov/docs/QPP_quality_measure_specifications/CQM-Measures/2019_Measure_001_MIPSCQM.pdf
df_temp = pd.read_sql_query("""\
    select distinct p.Id, value as obs_hba1c_most_recent, date as obs_hba1c_most_recent_date
    from patients p
    left outer join observations o ON p.Id = o.patient
    where o.code in (
        '4548-4' -- Hemoglobin A1c/Hemoglobin.total in Blood
    )
    group by p.Id
    having max(date)
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')
df_temp['obs_hba1c_most_recent_date'] = df_temp['obs_hba1c_most_recent_date'].astype('datetime64[ns]') # convert from object to date

df = pd.merge(df, df_temp, on='id', how='outer')

In [28]:
# Cond: Hypertension: Step 1: Condition of Hypertension
# Source: https://qpp.cms.gov/docs/QPP_quality_measure_specifications/CQM-Measures/2019_Measure_236_MIPSCQM.pdf
df_temp = pd.read_sql_query("""\
    select distinct p.Id, 1 as cond_hyp, start as cond_hyp_start
    from patients p
    inner join conditions c on p.Id = c.PATIENT
    where c.stop is null -- Active condition
        and c.code in (
            '59621000' -- Hypertension
        )
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')
df_temp['cond_hyp_start'] = df_temp['cond_hyp_start'].astype('datetime64[ns]') # convert from object to date

df_temp['cond_hyp_start_dayssince'] = (pd.to_datetime(enddate) - pd.to_datetime(df_temp['cond_hyp_start'])).dt.days
df_temp['cond_hyp_start_dayssince'] = df_temp['cond_hyp_start_dayssince'].fillna(0).astype(int)


df = pd.merge(df, df_temp, on='id', how='outer')
df['cond_hyp'] = df['cond_hyp'].fillna(0).astype(int)
df['cond_hyp'] = df['cond_hyp'].astype(bool)

In [29]:
# Obs Hyptertension: Step 2: Observation of blood pressure: Diastolic
df_temp = pd.read_sql_query("""\
    select p.Id, value as obs_bp_dia_most_recent, date as obs_bp_dia_most_recent_date
    from patients p
    left outer join observations o on p.Id = o.PATIENT
    where o.code in (
        '8462-4' -- Diastolic Blood Pressure
    ) 
    group by p.Id, o.code
    having max(o.Date)
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')
df_temp['obs_bp_dia_most_recent_date'] = df_temp['obs_bp_dia_most_recent_date'].astype('datetime64[ns]') # convert from object to date

df = pd.merge(df, df_temp, on='id', how='outer')

In [30]:
# Obs Hyptertension: Step 2: Observation of blood pressure: Systolic
df_temp = pd.read_sql_query("""\
    select p.Id, value as obs_bp_sys_most_recent, date as obs_bp_sys_most_recent_date
    from patients p
    left outer join observations o on p.Id = o.PATIENT
    where o.code in (
        '8480-6'  -- Systolic Blood Pressure
    )
    group by p.Id, o.code
    having max(o.Date)
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')
df_temp['obs_bp_sys_most_recent_date'] = df_temp['obs_bp_sys_most_recent_date'].astype('datetime64[ns]') # convert from object to date

df = pd.merge(df, df_temp, on='id', how='outer')

In [31]:
# Cond: Mental Health
df_temp = pd.read_sql_query("""\
    select distinct p.Id, 1 as cond_mentalhealth, start as cond_mentalhealth_start
    from patients p
    inner join conditions c on p.Id = c.PATIENT
    where c.stop is null -- Active condition
        and c.code in (
            '80583007', -- Severe anxiety (panic) (finding
            '370143000', -- Major depression disorder
            '73595000' -- Stress (finding)
        )
    group by p.Id
    having min(start)
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')
df_temp['cond_mentalhealth_start'] = df_temp['cond_mentalhealth_start'].astype('datetime64[ns]') # convert from object to date

df_temp['cond_mentalhealth_start_dayssince'] = (pd.to_datetime(enddate) - pd.to_datetime(df_temp['cond_mentalhealth_start'])).dt.days
df_temp['cond_mentalhealth_start_dayssince'] = df_temp['cond_mentalhealth_start_dayssince'].fillna(0).astype(int)

df = pd.merge(df, df_temp, on='id', how='outer')
df['cond_mentalhealth'] = df['cond_mentalhealth'].fillna(0).astype(int)
df['cond_mentalhealth'] = df['cond_mentalhealth'].astype(bool)

In [32]:
# Cond: Substance (including tobacco)
df_temp = pd.read_sql_query("""\
    select distinct p.Id, 1 as cond_substance, start as cond_substance_start
    from patients p
    inner join conditions c on p.Id = c.PATIENT
    where c.stop is null -- Active condition
        and c.code in (
            '7200002',   -- Alcoholism
            '55680006',  -- Drug overdose
            '55680006',  -- Drug overdose (disorder)
            '449868002', -- Smokes tobacco daily
            '10939881000119104', -- Unhealthy alcohol drinking behavior (finding)
            '5602001'    -- Opioid abuse (disorder)
        )
    group by p.Id
    having min(start)
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')
df_temp['cond_substance_start'] = df_temp['cond_substance_start'].astype('datetime64[ns]') # convert from object to date

df_temp['cond_substance_start_dayssince'] = (pd.to_datetime(enddate) - pd.to_datetime(df_temp['cond_substance_start'])).dt.days
df_temp['cond_substance_start_dayssince'] = df_temp['cond_substance_start_dayssince'].fillna(0).astype(int)

df = pd.merge(df, df_temp, on='id', how='outer')
df['cond_substance'] = df['cond_substance'].fillna(0).astype(int)
df['cond_substance'] = df['cond_substance'].astype(bool)

In [33]:
# Cond: Asthma
df_temp = pd.read_sql_query("""\
    select distinct p.Id, 1 as cond_asthma, start as cond_asthma_start
    from patients p
    inner join conditions c on p.Id = c.PATIENT
    where c.stop is null -- Active condition
        and c.code in (
            '195967001' -- Asthma
        )
    group by p.Id
    having min(start)
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')
df_temp['cond_asthma_start'] = df_temp['cond_asthma_start'].astype('datetime64[ns]') # convert from object to date

df_temp['cond_asthma_start_dayssince'] = (pd.to_datetime(enddate) - pd.to_datetime(df_temp['cond_asthma_start'])).dt.days
df_temp['cond_asthma_start_dayssince'] = df_temp['cond_asthma_start_dayssince'].fillna(0).astype(int)

df = pd.merge(df, df_temp, on='id', how='outer')
df['cond_asthma'] = df['cond_asthma'].fillna(0).astype(int)
df['cond_asthma'] = df['cond_asthma'].astype(bool)

In [34]:
# Cond: Cardiac
df_temp = pd.read_sql_query("""\
    select distinct p.Id, 1 as cond_cardiac, start as cond_cardiac_start
    from patients p
    inner join conditions c on p.Id = c.PATIENT
    where c.stop is null -- Active condition
        and c.code in (
            '49436004', -- Atrial Fibrillation
            '410429000', -- Cardiac Arrest
            '88805009', -- Chronic congestive heart failure (disorder)
            '53741008', -- Coronary Heart Disease
            '429007001', -- History of cardiac arrest (situation)
            '22298006', -- Myocardial Infarction
            '399211009' -- History of myocardial infarction (situation)
        )
    group by p.Id
    having min(start)
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')
df_temp['cond_cardiac_start'] = df_temp['cond_cardiac_start'].astype('datetime64[ns]') # convert from object to date

df_temp['cond_cardiac_start_dayssince'] = (pd.to_datetime(enddate) - pd.to_datetime(df_temp['cond_cardiac_start'])).dt.days
df_temp['cond_cardiac_start_dayssince'] = df_temp['cond_cardiac_start_dayssince'].fillna(0).astype(int)

df = pd.merge(df, df_temp, on='id', how='outer')
df['cond_cardiac'] = df['cond_cardiac'].fillna(0).astype(int)
df['cond_cardiac'] = df['cond_cardiac'].astype(bool)

In [35]:
# Cond: Cancer
df_temp = pd.read_sql_query("""\
    select distinct p.Id, 1 as cond_cancer, start as cond_cancer_start
    from patients p
    inner join conditions c on p.Id = c.PATIENT
    where c.stop is null -- Active condition
        and c.code in (
            '92691004', -- Carcinoma in situ of prostate (disorder)
            '254837009', -- Malignant neoplasm of breast (disorder)
            '363406005', -- Malignant tumor of colon
            '424132000', -- Non-small cell carcinoma of lung  TNM stage 1 (disorder)
            '254637007', -- Non-small cell lung cancer (disorder)
            '109838007', -- Overlapping malignant neoplasm of colon
            '93761005', -- Primary malignant neoplasm of colon
            '162573006' -- Suspected lung cancer (situation)
        )
    group by p.Id
    having min(start)
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')
df_temp['cond_cancer_start'] = df_temp['cond_cancer_start'].astype('datetime64[ns]') # convert from object to date

df_temp['cond_cancer_start_dayssince'] = (pd.to_datetime(enddate) - pd.to_datetime(df_temp['cond_cancer_start'])).dt.days
df_temp['cond_cancer_start_dayssince'] = df_temp['cond_cancer_start_dayssince'].fillna(0).astype(int)

df = pd.merge(df, df_temp, on='id', how='outer')
df['cond_cancer'] = df['cond_cancer'].fillna(0).astype(int)
df['cond_cancer'] = df['cond_cancer'].astype(bool)

In [36]:
# Cond: Kidney
df_temp = pd.read_sql_query("""\
    select distinct p.Id, 1 as cond_kidney, start as cond_kidney_start
    from patients p
    inner join conditions c on p.Id = c.PATIENT
    where c.stop is null -- Active condition
        and c.code in (
            '431855005', -- Chronic kidney disease stage 1 (disorder)
            '431856006', -- Chronic kidney disease stage 2 (disorder)
            '433144002', -- Chronic kidney disease stage 3 (disorder)
            '431857002', -- Chronic kidney disease stage 4 (disorder)
            '46177005' -- End stage renal disease (disorder)
        )
    group by p.Id
    having min(start)
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')
df_temp['cond_kidney_start'] = df_temp['cond_kidney_start'].astype('datetime64[ns]') # convert from object to date

df_temp['cond_kidney_start_dayssince'] = (pd.to_datetime(enddate) - pd.to_datetime(df_temp['cond_kidney_start'])).dt.days
df_temp['cond_kidney_start_dayssince'] = df_temp['cond_kidney_start_dayssince'].fillna(0).astype(int)

df = pd.merge(df, df_temp, on='id', how='outer')
df['cond_kidney'] = df['cond_kidney'].fillna(0).astype(int)
df['cond_kidney'] = df['cond_kidney'].astype(bool)

In [37]:
# Cond: Stroke
df_temp = pd.read_sql_query("""\
    select distinct p.Id, 1 as cond_stroke, start as cond_stroke_start
    from patients p
    inner join conditions c on p.Id = c.PATIENT
    where c.stop is null -- Active condition
        and c.code in (
            '230690007' -- Stroke
        )
    group by p.Id
    having min(start)
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')
df_temp['cond_stroke_start'] = df_temp['cond_stroke_start'].astype('datetime64[ns]') # convert from object to date

df_temp['cond_stroke_start_dayssince'] = (pd.to_datetime(enddate) - pd.to_datetime(df_temp['cond_stroke_start'])).dt.days
df_temp['cond_stroke_start_dayssince'] = df_temp['cond_stroke_start_dayssince'].fillna(0).astype(int)

df = pd.merge(df, df_temp, on='id', how='outer')
df['cond_stroke'] = df['cond_stroke'].fillna(0).astype(int)
df['cond_stroke'] = df['cond_stroke'].astype(bool)

In [38]:
# Cond: Sepsis
df_temp = pd.read_sql_query("""\
    select distinct p.Id, 1 as cond_sepsis, start as cond_sepsis_start
    from patients p
    inner join conditions c on p.Id = c.PATIENT
    where c.stop is null -- Active condition
        and c.code in (
            '91302008', -- Sepsis (disorder)
            '76571007' -- Septic shock (disorder)
        )
    group by p.Id
    having min(start)
""", con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')
df_temp['cond_sepsis_start'] = df_temp['cond_sepsis_start'].astype('datetime64[ns]') # convert from object to date

df_temp['cond_sepsis_start_dayssince'] = (pd.to_datetime(enddate) - pd.to_datetime(df_temp['cond_sepsis_start'])).dt.days
df_temp['cond_sepsis_start_dayssince'] = df_temp['cond_sepsis_start_dayssince'].fillna(0).astype(int)

df = pd.merge(df, df_temp, on='id', how='outer')
df['cond_sepsis'] = df['cond_sepsis'].fillna(0).astype(int)
df['cond_sepsis'] = df['cond_sepsis'].astype(bool)

## Create a total "_dayssince" 
- for all chronic conditions (for use in predictive models)

In [39]:
# Number of days (sum) for all _dayssince conditions

# Get all of the column names that are named as a cond_ with _dayssince
cols_dayssince = [col for col in df.columns if 'cond' in col and 'dayssince' in col]

# Create a total days column that adds up all of the 'cond' 'dayssince' columns
df['cond_total_dayssince'] = 0
for col in cols_dayssince:
    df['cond_total_dayssince'] = df['cond_total_dayssince'] + df[col].fillna(0).astype(int)

# Cost Measures

In [40]:
# Total claim cost per patient (no wellness; within 2 years)
#  Note: I have gone back and forth on encounter cost vs. total claim cost, but total claim cost is more comprehensive, so I went with that.
#  Note: I only focus on high cost encounters, as a lot of the lower cost encounters
#  are preventative or check-ups in this particular dataset
#  Note: Ambulatory is NOT excluded as it contains a lot of prenatal and other visits with high total claims (and high procedure costs)
sql = f"""
    select p.Id, SUM(e.total_claim_cost) as cost_total_2yr
    from patients as p 
    left outer join encounters e on p.Id = e.patient
    where (julianday('{enddate}') - julianday(start) <= 730) -- 2 years or less
           and encounterclass not in ('wellness', 'outpatient', 'virtual')
    group by p.Id
"""

df_temp = pd.read_sql_query(sql, con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')

df = pd.merge(df, df_temp, on='id', how='outer')
df['cost_total_2yr'] = df['cost_total_2yr'].fillna(0).astype(int)
df['cost_total_2yr'] = df['cost_total_2yr'].round(0)

In [41]:
# Fill missing values with zeros (as some patients will have zero claims in two year period)
df['cost_total_2yr'] = df['cost_total_2yr'].fillna(0).astype(int)

In [42]:
# Highest cost encounter reason 
#  Note: I only focus on high cost encounters, as a lot of the lower cost encounters
#  are preventative or check-ups in this particular dataset
sql = f"""
    select patient as Id, coalesce(description, 'Unknown') || ': ' || coalesce(REASONDESCRIPTION, 'Unknown') as cost_enc_2yr_highcost_reason, total_claim_cost as cost_enc_2yr_highcost_cost
    from encounters
    where ((julianday('{enddate}') - julianday(start) <= 730) or stop is null)
        and encounterclass not in ('wellness', 'outpatient', 'virtual')
    group by patient
    having max(total_claim_cost)
"""

df_temp = pd.read_sql_query(sql, con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')

# Make the reason lowercase to keep consistent later
df_temp['cost_enc_2yr_highcost_reason'] = df_temp['cost_enc_2yr_highcost_reason'].str.lower()

df = pd.merge(df, df_temp, on='id', how='outer')

# replace missing with unknown (means that a reason is not available or have not had a non-well encounter recently)
df['cost_enc_2yr_highcost_reason'] = df['cost_enc_2yr_highcost_reason'].fillna('unknown: unknown').astype('str')

In [43]:
# Highest cost procedures reason
sql = f"""
    select id, coalesce(description, 'Unknown') || ': ' || coalesce(REASONDESCRIPTION, 'Unknown') as reason, numofproc, max(sum_base_procedure_cost) as cost
    from (
        select patient as Id, code, description, reasondescription, count(patient) as numofproc, sum(base_cost) as sum_base_procedure_cost
        from procedures
        where ((julianday('{enddate}') - julianday(start) <= 730) or stop is null)
        group by patient, code
    )
    group by id
    order by id
"""

df_temp = pd.read_sql_query(sql, con)

df_temp = df_temp.rename(columns=str.lower)
df_temp = df_temp.set_index('id')

# Rename
df_temp.rename(columns={'numofproc':'cost_proc_2yr_highcost_count', 
                        'reason':'cost_proc_2yr_highcost_reason', 
                        'cost': 'cost_proc_2yr_highcost_sumcost'},
               inplace=True)

# Make the reason lowercase to keep consistent later
df_temp['cost_proc_2yr_highcost_reason'] = df_temp['cost_proc_2yr_highcost_reason'].str.lower()

df = pd.merge(df, df_temp, on='id', how='outer')

# replace missing with unknown (means that a reason is not available)
df['cost_proc_2yr_highcost_reason'] = df['cost_proc_2yr_highcost_reason'].fillna('unknown: unknown').astype('str')
df['cost_proc_2yr_highcost_count'] = df['cost_proc_2yr_highcost_count'].fillna(0).astype(int)
df['cost_proc_2yr_highcost_sumcost'] = df['cost_proc_2yr_highcost_sumcost'].fillna(0).astype(int)

## Output

In [44]:
df.shape

(22632, 90)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22632 entries, 53e40e98-c764-53a4-aaf6-6318a3c3c95d to 9fdfb702-0f46-8899-fe8c-363733532bb6
Data columns (total 90 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   first                                      22632 non-null  object        
 1   last                                       22632 non-null  object        
 2   city                                       22632 non-null  object        
 3   state                                      22632 non-null  object        
 4   county                                     22632 non-null  object        
 5   fips                                       22632 non-null  int64         
 6   lat                                        22632 non-null  float64       
 7   lon                                        22632 non-null  float64       
 8   birthdate                          

In [46]:
df.to_csv('w3e1_patient_dataset.csv')